In [43]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Load dataset
dataset = pd.read_csv('/content/capsfinal.csv')

# Preprocessing
# Encode categorical features
restaurant_ids = dataset['Nama Resto'].astype('category').cat.codes.values
region_ids = dataset['daerah'].astype('category').cat.codes.values
menu_ids = dataset['menu'].astype('category').cat.codes.values

# Example of generating user interaction data (clicks)
user_clicks = np.random.randint(0, len(np.unique(restaurant_ids)), size=(1000,))
user_regions = np.random.randint(0, len(np.unique(region_ids)), size=(1000,))

# Split data into train and test sets
train_data, test_data = train_test_split(np.column_stack([user_clicks, user_regions]), test_size=0.2, random_state=42)

# Model
num_restaurants = len(np.unique(restaurant_ids))
num_regions = len(np.unique(region_ids))

restaurant_input = tf.keras.layers.Input(shape=(1,), name='restaurant_input', dtype='int32')
region_input = tf.keras.layers.Input(shape=(1,), name='region_input', dtype='int32')

restaurant_embedding = tf.keras.layers.Embedding(input_dim=num_restaurants, output_dim=50, name='restaurant_embedding')(restaurant_input)
region_embedding = tf.keras.layers.Embedding(input_dim=num_regions, output_dim=50, name='region_embedding')(region_input)

concat_embeddings = tf.keras.layers.Concatenate()([restaurant_embedding, region_embedding])
flatten_embeddings = tf.keras.layers.Flatten()(concat_embeddings)

dense_layer = tf.keras.layers.Dense(128, activation='relu')(flatten_embeddings)
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(dense_layer)

model = tf.keras.models.Model(inputs=[restaurant_input, region_input], outputs=output_layer)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit([train_data[:, 0], train_data[:, 1]], np.ones(train_data.shape[0]), epochs=10, batch_size=32)

# Save the model
model.save('rekomendasi.h5')


Epoch 1/10
25/25 [==============================] - 1s 2ms/step - loss: 0.5991 - accuracy: 0.9488
Epoch 2/10
25/25 [==============================] - 0s 2ms/step - loss: 0.2931 - accuracy: 1.0000
Epoch 3/10
25/25 [==============================] - 0s 3ms/step - loss: 0.0516 - accuracy: 1.0000
Epoch 4/10
25/25 [==============================] - 0s 2ms/step - loss: 0.0096 - accuracy: 1.0000
Epoch 5/10
25/25 [==============================] - 0s 2ms/step - loss: 0.0042 - accuracy: 1.0000
Epoch 6/10
25/25 [==============================] - 0s 2ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 7/10
25/25 [==============================] - 0s 2ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 8/10
25/25 [==============================] - 0s 2ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 9/10
25/25 [==============================] - 0s 3ms/step - loss: 9.9936e-04 - accuracy: 1.0000
Epoch 10/10
25/25 [==============================] - 0s 3ms/step - loss: 7.9022e-04 - accuracy: 1.0000


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [44]:
import pandas as pd
import numpy as np
import tensorflow as tf

# Load dataset
dataset = pd.read_csv('/content/capsfinal.csv')

# Check columns in dataset
print(dataset.columns)

# Encode categorical features
dataset['restaurant_id'] = dataset['Nama Resto'].astype('category').cat.codes
dataset['region_id'] = dataset['daerah'].astype('category').cat.codes

# Load the trained model
model = tf.keras.models.load_model('/content/restaurant_recommendation_model.h5')

# Function to recommend restaurants
def recommend_restaurants(clicked_restaurant_id, clicked_region_id, top_n=5):
    # Prepare input data for the model
    restaurant_ids = np.array([clicked_restaurant_id] * len(dataset))
    region_ids = np.array([clicked_region_id] * len(dataset))

    # Get prediction scores from the model
    predictions = model.predict([restaurant_ids, region_ids]).flatten()

    # Get the top N recommendations
    top_indices = predictions.argsort()[-top_n:][::-1]

    # Get the recommended restaurants' details
    recommended_restaurants = dataset.iloc[top_indices]
    return recommended_restaurants[['Nama Resto', 'alamat', 'daerah', 'rating', 'menu', 'review ']]

# Example user input
clicked_restaurant_id = 2 # Example restaurant ID (you can change it based on actual IDs)
clicked_region_id = 1  # Example region ID (you can change it based on actual IDs)

# Get recommendations
recommended_restaurants = recommend_restaurants(clicked_restaurant_id, clicked_region_id)

# Display recommendations
print(recommended_restaurants)


Index(['Nama Resto', 'alamat', 'daerah', 'buka_tutup', 'jam_buka', 'jam_tutup',
       'harga_semua', 'harga_awal', 'harga_akhir', 'rata_rata', 'rating',
       'telepon', 'tipe', 'menu', 'review '],
      dtype='object')
17/17 [==============================] - 0s 2ms/step
                              Nama Resto  \
538                Kopi@Kopi Ground Zero   
184        Martabak pecenongan 78 cawang   
170  TOUS les JOURS  -  Cibubur Junction   
171           Burger King Cibubur Square   
172                      Javakarta Resto   

                                                alamat         daerah  rating  \
538  Komplek Ruko Bukit Gading Indah, Blok Q No. 35...  Jakarta Utara     4.3   
184  Jl. Dewi Sartika No.21, RT.5/RW.12, Cawang, Ke...  Jakarta Timur     4.5   
170  JVJV+6MC, Jagorawi Toll Rd, RT.8/RW.7, Cibubur...  Jakarta Timur     3.8   
171  Jagorawi Toll Rd Jl. Rest Area KM X No.5, RT.7...  Jakarta Timur     4.5   
172  Javakarta Resto, Jl. Jambore No.6, RT.3/RW.13,... 